In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
import pandas as pd
from flaml import AutoML
import cftime
from sklearn.model_selection import train_test_split
import pyaf.ForecastEngine as autof
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from scipy import stats
from scipy.spatial.distance import jensenshannon
from sklearn.impute import KNNImputer
import pmdarima as pm
from pmdarima.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import statsmodels.api as sm
from multiprocessing import Pool, cpu_count
import time
from timeit import default_timer as timer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
def open_data():
    #Open the datasets
    
    cfl = xr.open_mfdataset("eth_cfl/*.nc", join='inner', compat='override')
    
    cfl["lon"] = np.arange(-180,180,2.5)
    cfl = cfl.sel(lat=slice(-60,60),lon=slice(-80,20))

    
    return cfl

In [ ]:
cfl = open_data()
cfl = cfl.drop("time_bnds")
cfl.lon

In [ ]:
cfl = cfl.assign(ua_50000=cfl["ua"].sel(plev=50000))
cfl = cfl.assign(va_50000=cfl["va"].sel(plev=50000))
cfl = cfl.drop("plev")
cfl = cfl.drop("ua")
cfl = cfl.drop("va")
#cfl = cfl.stack(location=['lat','lon'])
#print(cfl.values())
#cfl = cfl.to_array().transpose("time","lat","lon","variable")
cfl

In [ ]:
np.count_nonzero(np.isnan(cfl['va_50000'])) 
    

In [ ]:
#imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp= KNNImputer(n_neighbors=2, weights="uniform")
ua_flat = cfl['ua_100000'].to_numpy().reshape((120,2624))
print(np.shape(ua_flat))
#imp.fit(ua_flat)
#ua_impl = imp.transform(ua_flat)
ua_impl = imp.fit_transform(ua_flat)
print(np.shape(ua_impl))
ua_impl = ua_impl.reshape((120,64,41))

#imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = KNNImputer(n_neighbors=2, weights="uniform")
va_flat = cfl['va_100000'].to_numpy().reshape((120,2624))
#imp.fit(va_flat)
#va_impl = imp.transform(va_flat)
va_impl = imp.fit_transform(va_flat)
va_impl = va_impl.reshape((120,64,41))

In [ ]:
target = np.zeros((2624,5))
var_list = ['SST_cpl','ice_cov','psl', 'ua_60000', 'va_60000']

In [ ]:
X_train = np.arange('2006-01', '2016-01', dtype='datetime64[M]')
#X_test =  np.arange('2015-01', '2015-12', dtype='datetime64[M]')
X_pred =  np.arange('2020-01', '2021-01', dtype='datetime64[M]')

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(cfl['SST_cpl'])
sst_cpl_impl = imp.transform(cfl['SST_cpl'])
sst_cpl_impl = sst_cpl_impl.reshape((120,64,41))

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(cfl['ice_cov'])
ice_cov_impl = imp.transform(cfl['ice_cov'])
sst_cpl_impl = sst_cpl_impl.reshape((120,64,41))

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(cfl['SST_cpl'])
psl_impl = imp.transform(cfl['psl'])
psl_impl = psl_impl.reshape((120,64,41))

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(cfl['ua_100000'])
ua_impl = imp.transform(cfl['ua_100000'])
ua_impl = ua_impl.reshape((120,64,41))

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(cfl['va_100000'])
va_impl = imp.transform(cfl['va_100000'])
va_impl = va_impl.reshape((120,64,41))

In [ ]:
import threading
import time

exitFlag = 0

class myThread (threading.Thread):
   def __init__(self, threadID, name, counter):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.counter = counter
   def run(self):
      print("Starting " + self.name)
      print_time(self.name, 5, self.counter)
      print("Exiting " + self.name)

def print_time(threadName, counter, delay):
   while(counter):
      if exitFlag:
         threadName.exit()
      time.sleep(delay)
      print("%s: %s" % (threadName, time.ctime(time.time())))
      counter -= 1

# Create new threads
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

# Start new Threads
thread1.start()
thread2.start()

print("Exiting Main Thread")

In [ ]:
lats_per_thread = []
for i in range(0,64,4):
    lats_per_thread.append(np.array(cfl.lat)[i:i+4])
lats_per_thread

In [ ]:
results = np.zeros((64,41,60))

In [ ]:
results = []
for lat in np.array(cfl.lat):
    print(lat)
    print('===========')
    cur_lon = []
    for lon in np.array(cfl.lon):
        print(lon)
        print('----------')
        cur_loc_vars = []
        for var in var_list:
            print(var)
            y_train = np.array(cfl.sel(lat=lat,lon=lon)[var])
            model = pm.auto_arima(y_train, seasonal=True, m=12)
            res = model.predict(60)
            cur_loc_vars.append(res)
        cur_lon.append(cur_loc_vars)
    results.append(cur_lon)

In [ ]:
eng = autof.cForecastEngine()

df_train = pd.DataFrame({"Date": X_train,
   	                 "Signal": cfl['SST_cpl'].to_numpy().reshape(120,2624)  })
#eng.train(iInputDS=cfl['SST_cpl'], iTime='Date', iSignal='Signal', iHorizon=7)

In [ ]:
target = []
for i in range(2624):
    cur_idx = []
    for j in range(5):
        y_train = np.array(cfl.sel(location=cfl.location[i],variable=var_list[j]))
        automl = AutoML()
        automl.fit(X_train=X_train,  # a single column of timestamp
           y_train=y_train,  # value for each timestamp
           period=13,  # time horizon to forecast, e.g., 12 months
           task='ts_forecast', time_budget=1,  # time budget in seconds
           log_file_name="ts_forecast.log",
           eval_method="holdout",
          )
        cur_idx.append(automl.predict(X_test))
    print(cur_idx)
    break
    target.append(cur_idx)
        

In [ ]:
mod = sm.tsa.VARMAX(cfl['SST_cpl'].to_numpy().reshape(120,2624), order=(0,2), error_cov_type='diagonal')
res = mod.fit(maxiter=100, disp=False)

In [ ]:
res = mod.fit(maxiter=1000, disp=False)

In [ ]:
np.isnan(to_ret)

In [ ]:
p

In [ ]:
sst_cpl = cfl['SST_cpl'].to_numpy().reshape(120,2624)
ice_cov = cfl['ice_cov'].to_numpy().reshape(120,2624)
psl = cfl['psl'].to_numpy().reshape(120,2624)
ua_50k = cfl['ua_50000'].to_numpy().reshape(120,2624)
va_50k = cfl['va_50000'].to_numpy().reshape(120,2624)

In [ ]:
results_sst_cpl = np.zeros((64,41,60))
results_ice_cov = np.zeros((64,41,60))
results_psl = np.zeros((64,41,60))
results_ua_60k = np.zeros((64,41,60))
results_va_60k = np.zeros((64,41,60))

In [ ]:
lat_start = cfl.lat[0]
lon_start = cfl.lon[0]
lon_delta = cfl.lon[1] - cfl.lon[0]
lat_delta = cfl.lat[1] - cfl.lat[0]
lats_per_proc = []
for i in range(0,64,int(64/cpu_count())):
    lats_per_proc.append(np.array(cfl.lat)[i:i+int(64/cpu_count())])
lats_per_proc

In [ ]:
def work_func(lats):
    print(lats)
    for lat in lats:
        print(lat)
        print('===========')
        
        for lon in np.array(cfl.lon):
            print(lon)
            print('----------')
           
            
            y_train = np.array(cfl.sel(lat=lat,lon=lon)['SST_cpl'])
            model = pm.auto_arima(y_train, seasonal=True, m=12)
            res = model.predict(60)
            
            lat_idx = int((lat - lat_start)/lat_delta)
            lon_idx = int((lon - lon_start)/lon_delta) 
            for i in range(60):
                results_sst_cpl[lat_idx][lon_idx][i] = res[i]

In [ ]:
print(f'starting computations on {cpu_count()} cores')
start = timer()
with Pool() as pool:
    res = pool.map(work_func,lats_per_proc)
    print(res)
end = timer()
print(f'elapsed time: {end - start}')

In [ ]:
cfl = xr.open_mfdataset("eth_cfl/*.nc", join='inner', compat='override')
cfl["lon"] = np.arange(-180,180,2.5)
cfl = cfl.sel(lat=slice(-60,60),lon=slice(-80,20))cfl = cfl.assign(ua_60000=cfl["ua"].sel(plev=60000))
cfl = cfl.assign(va_60000=cfl["va"].sel(plev=60000))
cfl = cfl.mean("plev")
cfl = cfl.drop("ua")
cfl = cfl.drop("va")

lats_per_thread = []
for i in range(0,64,4):
    lats_per_thread.append(np.array(cfl.lat)[i:i+4])
lats_per_thread



In [ ]:
sst_cpl

In [ ]:
def split_sequence(sequence, look_back, forecast_horizon):
    X, y = list(), list()
    for i in range(len(sequence)): 
        lag_end = i + look_back
        forecast_end = lag_end + forecast_horizon
        if forecast_end > len(sequence):
            break
        seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
LOOK_BACK = 20
FORECAST_RANGE = 12
n_features = 2624
train_set = sst_cpl[:70]
test_set = sst_cpl[70:]
print(train_set.shape)
print(test_set.shape)
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [ ]:
early_stopping_callback = EarlyStoppingByLossVal(monitor='val_loss', value=10, verbose=1)
rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, mode='min', patience=3, min_lr=0.001)
epochs = 250
batch_size = 16
validation = 0.1

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)

In [ ]:
def eval_results(y_test,y_pred):
    print('R^2 Score: ' + str(r2_score(y_test, y_pred)))
    print('LSTM - RMSE: ' + str(mean_squared_error(y_test,y_pred, squared = False)))
    print('LSTM - MAE: ' + str(mean_absolute_error(y_test,y_pred)))
    plt.scatter(np.arange(len(y_test)), y_test,label='Observed')
    plt.scatter(np.arange(len(y_test)), y_pred,label='Predicted')
    plt.xlabel('Predicted Season')
    plt.ylabel('Predicted ACE'),
    plt.title('Testing Results')
    plt.legend()
    plt.show()
    plt.scatter(y_test,y_pred)
    plt.xlabel('Observed')
    plt.ylabel('Predicted')
    plt.title('Testing Results - Predicted v Observed')
    plt.show()
    plt.scatter(y_test, y_test-y_pred)
    plt.plot(y_test,np.zeros(len(y_test)))
    plt.xlabel('Observed')
    plt.ylabel('Residual from Predicted')
    plt.title('Testing Results - Residual')
    plt.show()

In [ ]:
def eval_results_metrics_only(y_test,y_pred):
    print('R^2 Score: ' + str(r2_score(y_test, y_pred)))
    print('LSTM - RMSE: ' + str(mean_squared_error(y_test,y_pred, squared = False)))
    print('LSTM - MAE: ' + str(mean_absolute_error(y_test,y_pred)))

In [ ]:
X_pred = sst_cpl[100:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
sst_cpl = np.concatenate((sst_cpl,y_next), axis=0)

In [ ]:
train_set = sst_cpl[:80]
test_set = sst_cpl[80:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

In [ ]:
X_pred = sst_cpl[112:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
sst_cpl = np.concatenate((sst_cpl,y_next), axis=0)
np.shape(sst_cpl)

In [ ]:
train_set = sst_cpl[:90]
test_set = sst_cpl[90:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

In [ ]:
X_pred = sst_cpl[124:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
sst_cpl= np.concatenate((sst_cpl,y_next), axis=0)
np.shape(sst_cpl)

In [ ]:
train_set = sst_cpl[:90]
test_set = sst_cpl[90:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = sst_cpl[136:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
sst_cpl= np.concatenate((sst_cpl,y_next), axis=0)
np.shape(sst_cpl)

In [ ]:
train_set = sst_cpl[:100]
test_set = sst_cpl[100:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = sst_cpl[148:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
sst_cpl_2020 = y_next.reshape((12,64,41))

In [ ]:
sst_cpl = cfl['SST_cpl'].to_numpy().reshape(120,2624)
LOOK_BACK = 25
FORECAST_RANGE = 20

In [ ]:
train_set = sst_cpl[:65]
test_set = sst_cpl[65:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = sst_cpl[95:].reshape((1,25,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(20,2624)

In [ ]:
sst_cpl= np.concatenate((sst_cpl,y_next), axis=0)
np.shape(sst_cpl)

In [ ]:
train_set = sst_cpl[:75]
test_set = sst_cpl[75:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = sst_cpl[115:].reshape((1,25,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(20,2624)

In [ ]:
sst_cpl= np.concatenate((sst_cpl,y_next), axis=0)
np.shape(sst_cpl)

In [ ]:
train_set = sst_cpl[:90]
test_set = sst_cpl[90:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = sst_cpl[135:].reshape((1,25,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(20,2624)

In [ ]:
sst_cpl= np.concatenate((sst_cpl,y_next), axis=0)
np.shape(sst_cpl)

In [ ]:
ice_cov = cfl['ice_cov'].to_numpy().reshape(120,2624)
psl = cfl['psl'].to_numpy().reshape(120,2624)

In [ ]:
ua_50k = cfl['ua_50000'].to_numpy().reshape(120,2624)
va_50k = cfl['va_50000'].to_numpy().reshape(120,2624)

In [ ]:
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

def split_sequence(sequence, look_back, forecast_horizon):
    X, y = list(), list()
    for i in range(len(sequence)): 
        lag_end = i + look_back
        forecast_end = lag_end + forecast_horizon
        if forecast_end > len(sequence):
            break
        seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

early_stopping_callback = EarlyStoppingByLossVal(monitor='val_loss', value=1, verbose=1)
rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, mode='min', patience=3, min_lr=0.001)
epochs = 250
batch_size = 16
validation = 0.1
LOOK_BACK = 20
FORECAST_RANGE = 12
n_features = 2624

In [ ]:
train_set = ice_cov[:70]
test_set = ice_cov[70:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = ice_cov[100:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ice_cov= np.concatenate((ice_cov,y_next), axis=0)
np.shape(ice_cov)

In [ ]:
train_set = ice_cov[:80]
test_set = ice_cov[80:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = ice_cov[112:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ice_cov= np.concatenate((ice_cov,y_next), axis=0)
np.shape(ice_cov)

In [ ]:
train_set = ice_cov[:90]
test_set = ice_cov[90:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = ice_cov[124:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ice_cov= np.concatenate((ice_cov,y_next), axis=0)
np.shape(ice_cov)

In [ ]:
train_set = ice_cov[:100]
test_set = ice_cov[100:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = ice_cov[136:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ice_cov= np.concatenate((ice_cov,y_next), axis=0)
np.shape(ice_cov)

In [ ]:
train_set = ice_cov[:110]
test_set = ice_cov[110:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = ice_cov[148:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ice_cov= np.concatenate((ice_cov,y_next), axis=0)
np.shape(ice_cov)

In [ ]:
train_set = ice_cov[:120]
test_set = ice_cov[120:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = ice_cov[160:].reshape((1,20,2624))
ice_cov_2020 = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
train_set = psl[:70]
test_set = psl[70:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = psl[100:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
psl= np.concatenate((psl,y_next), axis=0)
np.shape(psl)

In [ ]:
train_set = psl[:80]
test_set = psl[80:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = psl[112:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
psl= np.concatenate((psl,y_next), axis=0)
np.shape(psl)

In [ ]:
train_set = psl[:90]
test_set = psl[90:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = psl[124:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
psl= np.concatenate((psl,y_next), axis=0)
np.shape(psl)

In [ ]:
train_set = psl[:100]
test_set = psl[100:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = psl[136:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
psl= np.concatenate((psl,y_next), axis=0)
np.shape(psl)

In [ ]:
train_set = psl[:110]
test_set = psl[110:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())

X_pred = psl[148:].reshape((1,20,2624))
psl_2020 = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
psl= np.concatenate((psl,y_next), axis=0)
np.shape(psl)

In [ ]:
np.count_nonzero(np.isnan(ua_50k))

In [ ]:
train_set = ua_50k[:70]
test_set = ua_50k[70:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = ua_50k[100:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ua_50k= np.concatenate((ua_50k,y_next), axis=0)
np.shape(ua_50k)

In [ ]:
train_set = ua_50k[:80]
test_set = ua_50k[80:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = ua_50k[112:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ua_50k= np.concatenate((ua_50k,y_next), axis=0)
np.shape(ua_50k)

In [ ]:
train_set = ua_50k[:90]
test_set = ua_50k[90:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = ua_50k[124:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ua_50k= np.concatenate((ua_50k,y_next), axis=0)
np.shape(ua_50k)

In [ ]:
train_set = ua_50k[:100]
test_set = ua_50k[100:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = ua_50k[136:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ua_50k= np.concatenate((ua_50k,y_next), axis=0)
np.shape(ua_50k)

In [ ]:
train_set = ua_50k[:110]
test_set = ua_50k[110:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = ua_50k[148:].reshape((1,20,2624))
ua_50k_2020 = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
ua_50k= np.concatenate((ua_50k,ua_50k_2020), axis=0)
np.shape(ua_50k)

In [ ]:
train_set = va_50k[:70]
test_set = va_50k[70:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = va_50k[100:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
va_50k= np.concatenate((va_50k,y_next), axis=0)
np.shape(va_50k)

In [ ]:
train_set = va_50k[:80]
test_set = va_50k[80:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = va_50k[112:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
va_50k= np.concatenate((va_50k,y_next), axis=0)
np.shape(va_50k)

In [ ]:
train_set = va_50k[:90]
test_set = va_50k[90:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = va_50k[124:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
va_50k= np.concatenate((va_50k,y_next), axis=0)
np.shape(va_50k)

In [ ]:
train_set = va_50k[:100]
test_set = va_50k[100:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = va_50k[136:].reshape((1,20,2624))
y_next = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
va_50k= np.concatenate((va_50k,y_next), axis=0)
np.shape(va_50k)

In [ ]:
train_set = va_50k[:110]
test_set = va_50k[110:]
X_train, y_train = split_sequence(train_set, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(test_set,LOOK_BACK,FORECAST_RANGE)

In [ ]:
model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')

In [ ]:
history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])

In [ ]:
y_pred = model_enc_dec.predict(X_test, verbose=0)
eval_results_metrics_only(y_test.flatten(),y_pred.flatten())
X_pred = va_50k[148:].reshape((1,20,2624))
va_50k_2020 = model_enc_dec.predict(X_pred, verbose=0).reshape(12,2624)

In [ ]:
va_50k= np.concatenate((va_50k,va_50k_2020), axis=0)
np.shape(va_50k)

In [ ]:
print(np.shape(sst_cpl_2020))
print(np.shape(ice_cov_2020))
print(np.shape(psl_2020))
print(np.shape(ua_50k_2020))
print(np.shape(va_50k_2020))
sst_cpl_2020 = sst_cpl_2020.reshape((12,2624))

In [ ]:
target = np.zeros((12,13120))

In [ ]:
for m in range(12):
    for i in range(0,2624):
        target[m][i*5] = sst_cpl_2020[m][i] 
        target[m][i*5+1] = ice_cov_2020[m][i] 
        target[m][i*5+2] = psl_2020[m][i] 
        target[m][i*5+3] = ua_50k_2020[m][i] 
        target[m][i*5+4] = va_50k_2020[m][i] 
    

In [ ]:
np.savetxt("ctf_ensemble_0.txt",target)

In [ ]:
loaded = np.loadtxt("ctf_ensemble_0.txt")

In [ ]:
sst_cpl_returned = np.zeros((12,2624))
ice_cov_returned_cpl_returned = np.zeros((12,2624))
psl_returned = np.zeros((12,2624))
ua_50k_returned = np.zeros((12,2624))
va_50k_returned = np.zeros((12,2624))

for m in range(12):
    for i in range(0,13120,5):
        sst_cpl_returned[m][int(i/5)] = loaded[m][i]
        ice_cov_returned[m][int(i/5)] = loaded[m][i+1]
        psl_returned[m][int(i/5)] = loaded[m][i+2] 
        ua_50k_returned[m][int(i/5)] = loaded[m][i+3]
        va_50k_returned[m][int(i/5)] = loaded[m][i+4]

In [ ]:
print(np.shape(sst_cpl))
print(np.shape(sst_cpl_returned))

In [ ]:
sst_min = min(np.min(sst_cpl_returned),np.min(sst_cpl))
sst_max = min(np.max(sst_cpl_returned),np.max(sst_cpl))
ice_min = min(np.min(ice_cov_returned),np.min(ice_cov))
ice_max = min(np.max(ice_cov_returned),np.max(ice_cov))
p_min = min(np.min(psl_returned),np.min(psl))
p_max = min(np.max(psl_returned),np.max(psl))
ua_min = min(np.min(ua_50k_returned),np.min(ua_50k))
ua_max = min(np.max(ua_50k_returned),np.max(ua_50k))
va_min = min(np.min(va_50k_returned),np.min(va_50k))
va_max = min(np.max(va_50k_returned),np.max(va_50k))
X_sst = np.linspace(sst_min,sst_max)
X_ice = np.linspace(ice_min,ice_max)
X_p = np.linspace(p_min,p_max)
X_ua = np.linspace(ua_min,ua_max)
X_va = np.linspace(va_min,va_max)

In [ ]:
ext_sst_pdf = stats.gaussian_kde(sst_cpl_returned.flatten())
ctf_sst_pdf = stats.gaussian_kde(sst_cpl.flatten())
ext_ice_pdf = stats.gaussian_kde(ice_cov_returned.flatten())
ctf_ice_pdf = stats.gaussian_kde(ice_cov.flatten())
ext_p_pdf = stats.gaussian_kde(psl_returned.flatten())
ctf_p_pdf = stats.gaussian_kde(psl.flatten())
ext_ua_pdf = stats.gaussian_kde(ua_50k_returned.flatten())
ctf_ua_pdf = stats.gaussian_kde(ua_50k.flatten())
ext_va_pdf = stats.gaussian_kde(va_50k_returned.flatten())
ctf_va_pdf = stats.gaussian_kde(va_50k.flatten())

In [ ]:
y_sst_ext = ext_sst_pdf(X_sst)
y_sst_ctf = ctf_sst_pdf(X_sst)
y_ice_ext = ext_ice_pdf(X_ice)
y_ice_ctf = ctf_ice_pdf(X_ice)
y_p_ext = ext_p_pdf(X_p)
y_p_ctf = ctf_p_pdf(X_p)
y_ua_ext = ext_ua_pdf(X_ua)
y_ua_ctf = ctf_ua_pdf(X_ua)
y_va_ext = ext_va_pdf(X_va)
y_va_ctf = ctf_va_pdf(X_va)

In [ ]:
print('JS SST: ' + str(jensenshannon(y_sst_ext,y_sst_ctf)))
print('JS ice: ' + str(jensenshannon(y_ice_ext,y_ice_ctf)))
print('JS pressure: ' + str(jensenshannon(y_p_ext,y_p_ctf)))
print('JS ua: ' + str(jensenshannon(y_ua_ext,y_ua_ctf)))
print('JS va: ' + str(jensenshannon(y_va_ext,y_va_ctf)))